# CIC0193 - Fundamentos de Sistemas Inteligentes -- Projeto 2

---


##Prof. Díbio 

#####Aluno: *Vitor Vasconelos de Oliveira*
#####Matrícula: *180114778*


## **Código utilizado para realizar a partição dos datsets:**

In [ ]:
#Monta drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# link para caminho padrão /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

%cd /mydrive/yolov4

/
Mounted at /content/gdrive
'Colab Notebooks'   images  'My Drive'	 XML2TXT.ipynb	 yolov4
/content/gdrive/My Drive/yolov4


In [ ]:
import glob, os

# Current directory
#current_dir = '/content/gdrive/MyDrive/yolov4/darknet'

#print(current_dir)

current_dir = '/content/gdrive/MyDrive/yolov4/darknet/data/obj'

# Create and/or truncate train.txt and test.txt
file_train = open('/content/gdrive/MyDrive/yolov4/darknet/data/train.txt', 'w')
file_valid = open('/content/gdrive/MyDrive/yolov4/darknet/data/valid.txt', 'w')
file_test = open('/content/gdrive/MyDrive/yolov4/darknet/data/test.txt', 'w')

# Populate train.txt and test.txt
counter = 1
#index_test = round(100 / percentage_test)
for pathAndFilename in glob.iglob(os.path.join(current_dir, "*.jpg")):
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter <=13:
        counter += 1
        file_train.write("data/obj" + "/" + title + '.jpg' + "\n")
    elif counter == 14:
        file_valid.write("data/obj" + "/" + title + '.jpg' + "\n")
        counter += 1
    elif counter == 15:
        file_test.write("data/obj" + "/" + title + '.jpg' + "\n")
        counter = 1
      

## **Código utilizado para transferencia de imagens pelas pastas dos trives quando necessário:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
from os import listdir
from os.path import isfile, join, basename

def move(path_origem, path_destino, ext='jpg'):
    cont=0
    for item in [join(path_origem, f) for f in listdir(path_origem) if isfile(join(path_origem, f)) and f.endswith(ext)]:
        #print(item)
        shutil.move(item, join(path_destino, basename(item)))
        print('moved "{}" -> "{}"'.format(item, join(path_destino, basename(item))))
        cont+=1
    print(cont)

In [ ]:
move('/content/drive/MyDrive/images/low_abundance/images','/content/drive/MyDrive/yolov4/darknet/data/obj')
# move('/content/drive/MyDrive/images/super_abundance/images','/content/drive/MyDrive/yolov4/darknet/data/obj')
# move('/content/drive/MyDrive/images/moderate_abundance/images','/content/drive/MyDrive/yolov4/darknet/data/obj')

moved "/content/drive/MyDrive/images/low_abundance/images/IMG_20190218_231642_2.jpg" -> "/content/drive/MyDrive/yolov4/darknet/data/obj/IMG_20190218_231642_2.jpg"
moved "/content/drive/MyDrive/images/low_abundance/images/IMG_20190218_231855_0.jpg" -> "/content/drive/MyDrive/yolov4/darknet/data/obj/IMG_20190218_231855_0.jpg"
moved "/content/drive/MyDrive/images/low_abundance/images/IMG_20190218_231025_8.jpg" -> "/content/drive/MyDrive/yolov4/darknet/data/obj/IMG_20190218_231025_8.jpg"
moved "/content/drive/MyDrive/images/low_abundance/images/IMG_20190218_231327_7.jpg" -> "/content/drive/MyDrive/yolov4/darknet/data/obj/IMG_20190218_231327_7.jpg"
moved "/content/drive/MyDrive/images/low_abundance/images/IMG_20190218_231101_2.jpg" -> "/content/drive/MyDrive/yolov4/darknet/data/obj/IMG_20190218_231101_2.jpg"
moved "/content/drive/MyDrive/images/low_abundance/images/IMG_20190218_231601_4.jpg" -> "/content/drive/MyDrive/yolov4/darknet/data/obj/IMG_20190218_231601_4.jpg"
moved "/content/drive/

## **Código utilizado para tranformar anotaçõe em .XML para .txt capazes de serem lidos pela YoloV4:**

In [ ]:
s_path = '/content/drive/MyDrive/images/super_abundance/'
# s_path = '/content/drive/MyDrive/images/low_abundance/'
# s_path = '/content/drive/MyDrive/images/moderate_abundance/'

In [ ]:
# Auqi levous-se enconta anotações com erros
# por exemplo "whitlfys" ao invés de "whiteflys" foram ajustadas para compreenderam a mesma classe ainda sim

import xml.etree.ElementTree as ET
import glob
import os
import json


def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = s_path + "annotation/"
output_dir = s_path + "labels/"
image_dir = s_path + "images/"

# create the labels folder (output directory)
#os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = 0
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result :
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))